Modified from https://lectures.quantecon.org/jl/optgrowth.html

In [2]:
using QuantEcon, Optim, Distributions
using Optim
using Distributions

In [3]:
dist = Normal(0.1,2)
@show typeof(dist)

typeof(dist) = Distributions.Normal{Float64}


Distributions.Normal{Float64}

In [29]:
#For calculating the expectation of a normally distributed variables.
#This sort of general utility could be added to the Distributions library, for example.
function expectation_operator(dist::Distributions.Normal; n_quad = 20)
    nodes, weights = qnwnorm(n_quad, mean(dist), var(dist)) #Gaussian quadrature weights
    return f -> dot(f.(nodes), weights) #Returns an operator for taking expectations
end

#Example
dist = Normal(0.1,2)
E = expectation_operator(dist)
var(dist) ≈ E(x -> x^2) - E(x -> x)^2 #i.e. calculate the variance

true

In [26]:
function bellman_operator(w::Vector, 
                          grid::Vector,
                          β::AbstractFloat, 
                          u::Function, 
                          f::Function, 
                            dist,
                          Tw::Vector = similar(w);
                          compute_policy::Bool = false)

    # === Apply linear interpolation to w === #
    w_func = LinInterp(grid, w)
    E = expectation_operator(dist)

    if compute_policy
        σ = similar(w)
    end

    # == set Tw[i] = max_c { u(c) + β E w(f(y  - c) z)} == #
    for (i, y) in enumerate(grid)
        objective(c) = - u(c) - β * E(ζ -> w_func.(f.(y - c) .* exp(ζ)))
        res = optimize(objective, 1e-10, y)

        if compute_policy
            σ[i] = res.minimizer
        end
        Tw[i] = - res.minimum
    end

    if compute_policy
        return Tw, σ
    else
        return Tw
    end
end

bellman_operator (generic function with 3 methods)

In [21]:
α = 0.4
β = 0.96
μ = 0
s = 0.1

c1 = log(1 - α * β) / (1 - β)
c2 = (μ + α * log(α * β)) / (1 - α)
c3 = 1 / (1 - β)
c4 = 1 / (1 - α * β)

# Utility 
u(c) = log(c)

u_prime(c) = 1 / c

# Deterministic part of production function
f(k) = k^α

f_prime(k) = α * k^(α - 1)

# True optimal policy
c_star(y) = (1 - α * β) * y

# True value function
v_star(y) = c1 + c2 * (c3 - c4) + c4 * log(y)

v_star (generic function with 1 method)

In [20]:
grid_max = 4         # Largest grid point
grid_size = 200      # Number of grid points
shock_size = 250     # Number of shock draws in Monte Carlo integral

grid_y = collect(linspace(1e-5, grid_max, grid_size))
shocks = exp.(μ + s * randn(shock_size))

250-element Array{Float64,1}:
 1.17882 
 1.03475 
 0.772314
 1.01998 
 1.04106 
 1.07967 
 0.969199
 1.02022 
 0.921195
 1.00937 
 0.837468
 0.911398
 1.01586 
 ⋮       
 0.936155
 0.799707
 1.0635  
 1.04838 
 1.01744 
 0.913255
 0.938978
 1.05238 
 1.30786 
 1.00912 
 0.977582
 0.8729  

In [17]:
using Plots
gr()

Plots.GRBackend()

In [27]:

w = bellman_operator(v_star.(grid_y),
                     grid_y,
                     β,
                     log,
                     k -> k^α,
                     Normal(μ,s^2)) #Shock distribution

plot(grid_y, w, lw=2, alpha=0.6, label="Tv^*")
plot!(grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="v^")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -50 
 
 
 -45 
 
 
 -40 
 
 
 -35 
 
 
 -30 
 
 
 -25 
 
<polyline clip-path="url(#clip6402)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,375.869 38.8912,128.92 41.6256,113.602 44.3601,104.614 47.0946,98.2584 49.829,93.321 52.5635,89.2858 55.298,85.8744 58.0324,82.918 60.7669,80.3092 
 63.5014,77.9785 66.2358,75.8723 68.9703,73.945 71.7047,72.1739 74.4392,70.5358 77.1737,69.0074 79.9081,67.5812 82.6426,66.2381 85.3771,64.975 88.1115,63.7773 
 90.846,62.6438 93.5805,61.5632 96.3149,60.535 99.0494,59.5505 101.784,58.6096 104.518,57.7062 107.253,56.8382 109.987,56.0038 112.722,55.1985 115.456,54.4226 
 118.191,53.6724 120.925,52.9466 123.66,52.2447 126.394,51.5637 129.128,50.9029 131.863,50.2621 134.597,49.6386 137.332,49.0322 140.066,48.4426 142.801,47.8677 
 145.535,47.3073 148.27,46.7614 151.004,46.2282 153.739,45.7074 156.473,45.1989 159.208,44.7018 161.942,44.2153 164.677,43.7395 167.411,43.2739 170.145,42.8177 
 172.88,42.3706 175.614,41.9326 178.349,41.5031 181.083,41.0815 183.818,40.6679 186.552,40.2621 189.287,39.8634 192.021,39.4717 194.756,39.087 197.49,38.7088 
 200.225,38.337 202.959,37.9713 205.693,37.6115 208.428,37.2576 211.162,36.9092 213.897,36.5662 216.631,36.2283 219.366,35.8957 222.1,35.5679 224.835,35.2449 
 227.569,34.9265 230.304,34.6127 233.038,34.3033 235.773,33.9982 238.507,33.6972 241.241,33.4002 243.976,33.1071 246.71,32.8179 249.445,32.5325 252.179,32.2506 
 254.914,31.9723 257.648,31.6975 260.383,31.426 263.117,31.1579 265.852,30.8928 268.586,30.6311 271.321,30.3723 274.055,30.1165 276.79,29.8637 279.524,29.6137 
 282.258,29.3665 284.993,29.122 287.727,28.8803 290.462,28.6411 293.196,28.4045 295.931,28.1704 298.665,27.9386 301.4,27.7095 304.134,27.4825 306.869,27.2579 
 309.603,27.0355 312.338,26.8154 315.072,26.5974 317.806,26.3816 320.541,26.1678 323.275,25.9561 326.01,25.7464 328.744,25.5387 331.479,25.3329 334.213,25.129 
 336.948,24.927 339.682,24.7267 342.417,24.5283 345.151,24.3316 347.886,24.1367 350.62,23.9435 353.354,23.752 356.089,23.562 358.823,23.3738 361.558,23.187 
 364.292,23.0019 367.027,22.8183 369.761,22.6362 372.496,22.4556 375.23,22.2765 377.965,22.0988 380.699,21.9224 383.434,21.7477 386.168,21.574 388.903,21.402 
 391.637,21.231 394.371,21.0616 397.106,20.8933 399.84,20.7264 402.575,20.5606 405.309,20.3961 408.044,20.2329 410.778,20.0707 413.513,19.9099 416.247,19.7501 
 418.982,19.5916 421.716,19.4341 424.451,19.2778 427.185,19.1225 429.919,18.9683 432.654,18.8152 435.388,18.6631 438.123,18.5122 440.857,18.3621 443.592,18.2132 
 446.326,18.0652 449.061,17.9182 451.795,17.7722 454.53,17.627 457.264,17.483 459.999,17.3397 462.733,17.1975 465.467,17.0561 468.202,16.9156 470.936,16.7761 
 473.671,16.6373 476.405,16.4995 479.14,16.3625 481.874,16.2264 484.609,16.0911 487.343,15.9565 490.078,15.8229 492.812,15.69 495.547,15.5579 498.281,15.4267 
 501.016,15.2961 503.75,15.1664 506.484,15.0374 509.219,14.9091 511.953,14.7816 514.688,14.6548 517.422,14.5287 520.157,14.4034 522.891,14.2788 525.626,14.1548 
 528.36,14.0316 531.095,13.909 533.829,13.7871 536.564,13.6659 539.298,13.5453 542.032,13.4254 544.767,13.3062 547.501,13.1875 550.236,13.0695 552.97,12.9522 
 555.705,12.8354 558.439,12.7193 561.174,12.6037 563.908,12.4888 566.643,12.3745 569.377,12.2607 572.112,12.1476 574.846,12.035 577.58,11.923 580.315,11.8116 
 
 "/>
<polyline clip-path="url(#clip6402)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,297.182 38.8912,128.905 41.6256,113.576 44.3601,104.607 47.0946,98.2437 49.829,93.3076 52.5635,89.2744 55.298,85.8644 58.0324,82.9105 60.7669,80.3049 
 63.5014,77.9741 66.2358,75.8656 68.9703,73.9408 71.7047,72.17 74.4392,70.5306 77.1737,69.0043 79.9081,67.5766 82.6426,66.2354 85.3771,64.9709 88.1115,63.7748 
 90.846,62.6401 93.5805,61.5607 96.314

In [ ]:
w = 5 * log.(grid_y)  # An initial condition -- fairly arbitrary
n = 35

lb = "initial condition"
plot(grid_y, w, lw=2, alpha=0.6, label=lb)
for i in 1:n
    w = bellman_operator(w,
                         grid_y,
                         β,
                         log,
                         k -> k^α,
                         shocks)

    plot!(grid_y, w, color=jet(i / n), lw=2, alpha=0.6)
end

lb = "true value function"
ax[:plot](grid_y, v_star.(grid_y), "k-", lw=2, alpha=0.8, label=lb)
ax[:legend](loc="lower right")

show()

In [ ]:
function solve_optgrowth(initial_w;
                         tol::AbstractFloat=1e-6,
                         max_iter::Integer=500)

    w = initial_w  # Set initial condition
    error = tol + 1
    i = 0

    # == Create storage array for bellman_operator. Reduces  memory
    # allocation and speeds code up == #
    Tw = similar(grid_y)

    # Iterate to find solution

    while (error > tol) && (i < max_iter)
        w_new = bellman_operator(w,
                             grid_y,
                             β,
                             log,
                             k -> k^α,
                             shocks)
        error = maximum(abs, w_new - w)
        w = w_new
        i += 1
    end

    return w
end

In [ ]:
initial_w = 5 * log.(grid_y)
v_star_approx = solve_optgrowth(initial_w)

fig, ax = subplots(figsize=(9, 5))
ax[:set_ylim](-35, -24)
ax[:plot](grid_y, v_star_approx, lw=2, alpha=0.6, label="approximate value function")
ax[:plot](grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="true value function")
ax[:legend](loc="lower right")
show()

In [ ]:
import QuantEcon: compute_fixed_point

Tw = similar(grid_y)
initial_w = 5 * log.(grid_y)

bellman_operator(w) = bellman_operator(w,
                                       grid_y,
                                       β,
                                       log,
                                       k -> k^α,
                                       shocks)

v_star_approx = compute_fixed_point(bellman_operator,
                                    initial_w,
                                    max_iter=500,
                                    verbose=2,
                                    print_skip=10,
                                    err_tol=1e-5)

```none
Compute iterate 10 with error 0.709153897728406
Compute iterate 20 with error 0.47095844432889145
Compute iterate 30 with error 0.3131085083453158
Compute iterate 40 with error 0.20816475495214704
Compute iterate 50 with error 0.13839472275577336
Compute iterate 60 with error 0.09200932833702069
Compute iterate 70 with error 0.061170804294146564
Compute iterate 80 with error 0.04066834706496181
Compute iterate 90 with error 0.027037644380378367
Compute iterate 100 with error 0.01797550838462314
Compute iterate 110 with error 0.011950704623018282
Compute iterate 120 with error 0.00794521845620011
Compute iterate 130 with error 0.005282240529794535
Compute iterate 140 with error 0.003511805895449527
Compute iterate 150 with error 0.0023347631704915273
Compute iterate 160 with error 0.001552226753144481
Compute iterate 170 with error 0.0010319710041208907
Compute iterate 180 with error 0.0006860880028760619
Compute iterate 190 with error 0.0004561336956570017
Compute iterate 200 with error 0.00030325256715002524
Compute iterate 210 with error 0.00020161220411551994
Compute iterate 220 with error 0.0001340383727708172
Compute iterate 230 with error 8.911308481529545e-5
Compute iterate 240 with error 5.924528747414115e-5
Compute iterate 250 with error 3.9388200878676116e-5
Compute iterate 260 with error 2.6186561168373146e-5
Compute iterate 270 with error 1.7409680953761608e-5
Compute iterate 280 with error 1.1574523867352582e-5
Converged in 284 steps
```


In [ ]:
fig, ax = subplots(figsize=(9, 5))
ax[:set_ylim](-35, -24)
ax[:plot](grid_y, v_star_approx, lw=2, alpha=0.6, label="approximate value function")
ax[:plot](grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="true value function")
ax[:legend](loc="lower right")
show()

In [ ]:
Tw, σ = bellman_operator(v_star_approx,
                         grid_y,
                         β,
                         log,
                         k -> k^α,
                         shocks;
                         compute_policy=true)


cstar = (1 - α * β) * grid_y

fig, ax = subplots(figsize=(9, 5))
ax[:plot](grid_y, σ, lw=2, alpha=0.6, label="approximate policy function")
ax[:plot](grid_y, cstar, lw=2, alpha=0.6, label="true policy function")
ax[:legend](loc="lower right")
show()

In [ ]:
s = 0.05
shocks = exp.(μ + s * randn(shock_size))

In [ ]:
"""
Compute a time series given consumption policy σ.
"""
function simulate_og(σ, y0 = 0.1, ts_length=100)
    y = Array{Float64}(ts_length)
    ξ = randn(ts_length-1)
    y[1] = y0
    for t in 1:(ts_length-1)
        y[t+1] = (y[t] - σ(y[t]))^α * exp(μ + s * ξ[t])
    end
    return y
end

fig, ax = subplots(figsize=(9, 6))

for β in (0.9, 0.94, 0.98)

    Tw = similar(grid_y)
    initial_w = 5 * log.(grid_y)

    v_star_approx = compute_fixed_point(bellman_operator,
                                        initial_w,
                                        max_iter=50,
                                        verbose=0,
                                        print_skip=10,
                                        err_tol=1e-5)

    Tw, σ = bellman_operator(v_star_approx,
                             grid_y,
                             β,
                             log,
                             k -> k^α,
                             shocks,
                             compute_policy=true)

    σ_func = LinInterp(grid_y, σ)
    y = simulate_og(σ_func)
    ax[:plot](y, lw=2, alpha=0.6, label="β = $β" )
end


ax[:legend](loc="lower right")
show()